In [4]:
import time
import os
import copy
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torchvision
from torchvision.io import read_image
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

class SatellitesDetectorDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir 
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform: 
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
# TESTING
def TestModel(model, dataloaders):
    prec = []
    acc = []
    rec = []
    f1 = []
    prec_s = 0
    rec_s = 0
    f1_s = 0

    all_targets = np.array([])
    all_predictions = np.array([])

    for batch_idx, (data,targets) in enumerate(dataloaders['test']):
        data = data.to(device="cuda")
        targets = targets.to(device="cuda")
        ## Forward Pass
        scores = model(data)
        _, predictions = scores.max(1)
        targets = targets.cpu()
        predictions = predictions.cpu()
        all_targets = np.concatenate((all_targets, targets.numpy()), axis=None)
        all_predictions = np.concatenate((all_predictions, predictions.numpy()), axis=None)

    prec = (metrics.precision_score(all_targets, all_predictions, average=None))
    acc = (metrics.accuracy_score(all_targets, all_predictions))
    rec = (metrics.recall_score(all_targets, all_predictions, average=None))
    f1 = (metrics.f1_score(all_targets, all_predictions, average=None))
    prec_s = metrics.precision_score(all_targets, all_predictions, average='macro')
    rec_s = metrics.recall_score(all_targets, all_predictions, average='macro')
    f1_s = metrics.f1_score(all_targets, all_predictions, average='macro')

    print("Macro metrics:");
    print(f"Precision = {prec_s}")
    print(f"Recall = {rec_s}")
    print(f"F1 = {f1_s}")
    print(f"Accuracy = {acc}")
    print("\nMetrics Details:")
    print(f"Precision = {prec}")
    print(f"Recall = {rec}")
    print(f"F1 = {f1}") 

# TRY Send the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
input_size = 224
# Transform settings
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Lambda(lambda image: image.convert('RGB')),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Lambda(lambda image: image.convert('RGB')),
        transforms.Resize(input_size), 
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Lambda(lambda image: image.convert('RGB')),
        transforms.Resize(input_size), 
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

#preparing the train, validation and test dataset
torch.manual_seed(43)
train_ds = SatellitesDetectorDataset(annotations_file='D:/Documents/CNN/Satellite Detector/SatellitesDataset/sd_labels.csv',
                                     img_dir='D:/Documents/CNN/Satellite Detector/SatellitesDataset/satellites_data_bn/', 
                                     transform=data_transforms['test'])
val_size = 100
test_size = 100
train_size = len(train_ds) - val_size - test_size
train_ds, val_ds, test_ds = random_split(train_ds, [train_size, val_size, test_size])
#val_ds.transform = data_transforms['val']
#test_ds.transform = data_transforms['test']

# Create training and validation datasets
#image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
image_datasets = {'train':train_ds, 'val':val_ds, 'test':test_ds}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1, shuffle=False) for x in ['train', 'val', 'test']}

cuda


In [5]:
# USING IT FOR THE TEST SET
model_ft2 = torch.load("sd_resnet152-10cls-bnimg.pth") 
model_ft2.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
TestModel(model_ft2,dataloaders_dict)

Macro metrics:
Precision = 0.9508333333333333
Recall = 0.9178535353535354
F1 = 0.9307578302106503
Accuracy = 0.93

Metrics Details:
Precision = [0.91666667 1.         1.         1.         1.         0.8
 0.875      1.         0.91666667 1.        ]
Recall = [1.         1.         0.90909091 0.9        0.875      1.
 0.77777778 1.         0.91666667 0.8       ]
F1 = [0.95652174 1.         0.95238095 0.94736842 0.93333333 0.88888889
 0.82352941 1.         0.91666667 0.88888889]
